In [1]:
import pandas as pd
import numpy as np
import urllib
import time # понадобится, чтобы вставлять паузы в выполнение кода

from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [2]:
driver = webdriver.Chrome('chromedriver.exe') # Откроем браузер через драйвер
driver.get('https://2gis.ru/astrakhan/') # В открывшемся браузере перейдем на сайт 2ГИС

In [3]:
# Определим поле для отправки поисковых запросов
search_bar = driver.find_element_by_xpath('//*[@id="root"]/div/div/div[1]/div[1]/div[2]/div/div/div[1]/div/div/div/div/div[2]/form/div/input')

In [4]:
# Выведем список заведений общепита
search_bar.send_keys('Поесть')
search_bar.send_keys(Keys.ENTER)

In [64]:
# Для переключения между страницами нужно понять, сколько их всего
num_of_pages = int(np.ceil(int(driver.find_element_by_xpath('//*[@id="root"]/div/div/div[1]/div[1]/div[2]/div/div/div[2]/div/div/div/div[1]/header/div[3]/div/div[1]/div/div/a/span').text) / 12))
print('Всего страниц организаций в поисковой выдаче: %s' % num_of_pages)

Всего страниц организаций в поисковой выдаче: 63


Переключение между организациями будет происходить через подставление порядкового номера в XPATH

Например:
* XPATH первой организации: //*[@id="root"]/div/.../div/div[1]/div/div[2]
* XPATH второй организации: //*[@id="root"]/div/.../div/div[2]/div/div[2]

In [14]:
# Обозначим некоторые постоянные
close_xp = '//*[@id="root"]/div/div/div[1]/div[1]/div[1]/div[2]/div/div[1]/div'
next_page_xp = '//*[@id="root"]/div/div/div[1]/div[1]/div[2]/div/div/div[2]/div/div/div/div[2]/div[2]/div[1]/div/div/div[1]/div[3]/div[2]/div[2]'

In [72]:
# Инициализируем таблицы
df_result = pd.DataFrame()
df_errors = pd.DataFrame()

In [75]:
for i in range(1, num_of_pages+1):
    try:
        for j in range(1, 13): # от 1 до 12
            try:
                name_xp = '//*[@id="root"]/div/div/div[1]/div[1]/div[2]/div/div/div[2]/div/div/div/div[2]/div[2]/div[1]/div/div/div[1]/div[2]/div/div[' + str(j) + ']/div/div[2]'
                company_card = driver.find_element_by_xpath(name_xp)
                company_card.click() # 1. Откроем карточку организации

                time.sleep(0.5)

                df_temp = pd.DataFrame({
                    'name': driver.find_element_by_tag_name('h1').text, # 2. Сохраним название организации
                    'address': driver.find_element_by_xpath("//a[contains(@href, '/astrakhan/geo/')]").text # 3. Сохраним адрес
                }, index=[0])

                driver.find_element_by_xpath(close_xp).click() # Закроем карточку

                df_result = df_result.append(df_temp)
                time.sleep(0.5)

            except Exception:
                df_temp = pd.DataFrame({'i': i, 'j': j}, index=[0])
                df_errors = df_errors.append(df_temp)

        driver.find_element_by_xpath(next_page_xp).click() # Переключимся на следующую страницу
        print(i)
        time.sleep(0.7) # Подождем, чтоб список прогрузился
    
    except Exception:
        pass

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="root"]/div/div/div[1]/div[1]/div[2]/div/div/div[2]/div/div/div/div[2]/div[2]/div[1]/div/div/div[1]/div[3]/div[2]/div[2]"}
  (Session info: chrome=85.0.4183.121)
  (Driver info: chromedriver=2.42.591088 (7b2b2dca23cca0862f674758c9a3933e685c27d5),platform=Windows NT 10.0.18362 x86_64)


In [76]:
print('Кол-во успешно записанных организаций: %s' % df_result.shape[0])
print('Кол-во ошибок: %s' % df_errors.shape[0])

Кол-во успешно записанных организаций: 769
Кол-во ошибок: 11


In [77]:
df_result.head()

,name,address
0,Pizza Funk,"Боевая, 83е"
0,Кофейня на Савушкина,"Савушкина, 24а"
0,MYBOX,"Ахматовская, 3"
0,Кинза,"Кремлёвская, 1а"
0,ШаурМен,"Максима Горького, 15"


In [78]:
df_errors.head()

,i,j
0,2,6
0,4,6
0,5,4
0,10,6
0,15,6


In [79]:
df_result.to_csv('addresses.csv', index=False)